In [1]:
from core.imports import *

/usr/local/lib/python3.6/dist-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509


In [2]:
path = fs.join(ETL_PATH, job_name, 'SIGIR-ecom-data-challenge/train')

# Populate EDBs

In [3]:
import csv

In [12]:
from core.embeddeddb2 import EmbeddedDB2

def prepare(it, key_field):
    for doc in it:
        yield doc[key_field], doc

In [13]:
edb = EmbeddedDB2(fs.join(path, 'sku_to_content.edb'))
it = csv.DictReader(open(fs.join(path, 'sku_to_content.csv')))
edb.put_bulk(progress(prepare(it, 'product_sku_hash')))
edb.close()

2022-08-12 19:40:40 pid 4855 | starting to iterate, updating every 1 seconds
2022-08-12 19:40:41 pid 4855 | 27440, speed: 37664.13 it/s, elapsed 0.73 seconds
2022-08-12 19:40:42 pid 4855 | 66386, speed: 38858.48 it/s, elapsed 1.71 seconds
2022-08-12 19:40:42 pid 4855 | finished . Total time: 1.71 seconds


In [10]:
sessions = pd.read_csv(fs.join(path, 'browsing_train.csv'))
sessions = sessions.sort_values('session_id_hash')
sessions.to_csv(fs.join(path, 'sorted_browsing_train.csv'), index=False)

In [16]:
from itertools import groupby

def browsing_iter():
    it = csv.DictReader(open(fs.join(path, 'sorted_browsing_train.csv')))
    git = groupby(it, key=lambda x: x['session_id_hash'])
    for session_id_hash, sess in git:
        sess = list(sess)
        for doc in sess:
            doc['server_timestamp_epoch_ms'] = int(doc['server_timestamp_epoch_ms'])
        sess.sort(key=lambda x: x['server_timestamp_epoch_ms'])
        yield session_id_hash, sess
   

In [17]:
                     
edb = EmbeddedDB2(fs.join(path, 'browsing_train.edb'))
edb.put_bulk(progress(browsing_iter()))
edb.close()

2022-08-18 13:37:40 pid 4855 | starting to iterate, updating every 1 seconds
2022-08-18 13:37:41 pid 4855 | 17608, speed: 15823.29 it/s, elapsed 1.11 seconds
2022-08-18 13:37:42 pid 4855 | 32971, speed: 15604.14 it/s, elapsed 2.11 seconds
2022-08-18 13:37:43 pid 4855 | 48383, speed: 15541.42 it/s, elapsed 3.11 seconds
2022-08-18 13:37:44 pid 4855 | 63497, speed: 15436.77 it/s, elapsed 4.11 seconds
2022-08-18 13:37:45 pid 4855 | 78232, speed: 15299.43 it/s, elapsed 5.11 seconds
2022-08-18 13:37:46 pid 4855 | 93060, speed: 15222.08 it/s, elapsed 6.11 seconds
2022-08-18 13:37:47 pid 4855 | 108022, speed: 15185.49 it/s, elapsed 7.11 seconds
2022-08-18 13:37:48 pid 4855 | 123247, speed: 15190.34 it/s, elapsed 8.11 seconds
2022-08-18 13:37:49 pid 4855 | 138540, speed: 15201.41 it/s, elapsed 9.11 seconds
2022-08-18 13:37:50 pid 4855 | print interval is now 5s
2022-08-18 13:37:54 pid 4855 | 213094, speed: 15098.45 it/s, elapsed 14.11 seconds
2022-08-18 13:37:59 pid 4855 | 285181, speed: 14920.